In [1]:
import pyautogui
import time
import webbrowser
import pandas as pd
import bs4
import pyperclip
import numpy as np
import os
import re
import math
from datetime import datetime, timedelta


from urllib.parse import urlparse
#%% Windows simulation
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.actions.action_builder import ActionBuilder
from selenium.common.exceptions import NoSuchElementException



In [21]:
df_venture = pd.read_csv('../Data/ico_manully.csv')
df_venture

,Unnamed: 0,Project,Abbr,Link,Sale Price,ROI,Total Raise,Launchpad,Ended,When,...,ICO Order,Social,Funds and Investors,Investor Count,ICO Start,ICO End,Soft Cap,Hard Cap,ICO Raised,Other Link
0,0,Artyfact,ARTY,https://cryptorank.io/ico/artyfact,0.300,NaN,2450000.0,TruePNL,2023/10/30,2022/10/11,...,1,"{'Website': 'https://artyfact.game/', 'X (Ex-T...",NaN,0,2023/8/1,2023/10/30,660000.0,1320000.0,1320000,https://foundico.com/ico/artyfact.html
1,1,SupraOracles,SUPRA,https://cryptorank.io/ico/supra-oracles,0.048,NaN,2400000.0,NaN,2023/10/26,2023/10/25,...,1,"{'Website': 'https://supraoracles.com/', 'X (E...","['Coinbase Ventures', 'HashKey Capital', 'Anim...",22,2023/10/25,2023/10/26,NaN,2400000.0,2400000,https://common.fund/projects/supra-public-sale
2,2,Islamic Coin,ISLM,https://cryptorank.io/ico/islamic-coin,0.300,0.60,2400000.0,Republic,2023/10/9,2023/9/12,...,1,"{'Website': 'https://islamiccoin.net/', 'X (Ex...",NaN,0,2023/9/12,2023/10/9,NaN,4300000.0,2399474,NaN
3,3,Archway,ARCH,https://cryptorank.io/ico/archway,0.200,0.38,6030000.0,Coinlist+1,2023/9/27,2023/6/15,...,1,"['https://archway.io/', 'https://twitter.com/a...","['Blockchain Capital', 'Hashed Fund\nLead', 'C...",12,2023/6/15,2023/6/23,6000000.0,27000000.0,27000000,NaN
4,4,ScapesMania,$MANIA,https://cryptorank.io/ico/scapesmania,0.009,NaN,5870000.0,NaN,2023/9/10,2023/8/28,...,2,{'Website': 'https://scapesmania.io/?utm_sourc...,NaN,0,2023/8/28,2023/9/10,NaN,720000.0,720000,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
789,789,Waves,WAVES,https://cryptorank.io/ico/waves,0.188,12.01,16000000.0,NaN,NaN,NaN,...,1,"{'Website': 'https://waves.tech/', 'X (Ex-Twit...",NaN,0,2016/4/11,2016/5/30,NaN,16000000.0,16000000,NaN
790,790,WeTrust,TRST,https://cryptorank.io/ico/trust,0.060,0.01,NaN,NaN,NaN,NaN,...,1,"{'Website': 'https://www.wetrust.io/', 'X (Ex-...",NaN,0,NaN,NaN,NaN,NaN,0,NaN
791,791,ZenGold,ZENGOLD,https://cryptorank.io/ico/zengold,3.860,NaN,NaN,NaN,NaN,NaN,...,1,"['http://www.zengold.org/', 'https://zengold.o...",NaN,0,NaN,NaN,NaN,NaN,0,NaN
792,792,Zipper,ZIP,https://cryptorank.io/ico/zip,0.030,0.00,NaN,NaN,NaN,NaN,...,1,"['http://zipper.io/', 'https://twitter.com/Zip...",NaN,0,NaN,NaN,NaN,NaN,0,NaN


In [41]:
df_venture['Success'] = df_venture['ICO Raised']>0
num = len(df_venture['ICO Raised'])
num_success = sum(df_venture['Success'])
num_nonsucc = sum(df_venture['ICO Raised']==0)

print(f'Total num: {num}; Success:{num_success}; Non-Success:{num_nonsucc}.')

num_soft = sum(~df_venture['Soft Cap'].isnull())
num_hard = sum(~df_venture['Hard Cap'].isnull())
num_reach = sum(df_venture['Hard Cap']<=df_venture['ICO Raised'])

print(f'Soft cap: {num_soft}, Hard cap: {num_hard}.Reach :{num_reach}')

Total num: 794; Success:579; Non-Success:215.
Soft cap: 24, Hard cap: 547.Reach :413


In [40]:
def date_clean(date_str):
     date_str = str(date_str)
     if len(date_str) == 0:
        return None
     try:
          
          return datetime.strptime(date_str, "%Y/%m/%d")
     except ValueError:
          
          return None
     
df_venture['ICO End'] = df_venture['ICO End'].apply(date_clean)
df_venture['ICO Start'] = df_venture['ICO Start'].apply(date_clean)

In [42]:
df_venture['ICO Days'] = df_venture['ICO End']-df_venture['ICO Start']+ timedelta(days=1)
df_venture.columns


Index(['Unnamed: 0', 'Project', 'Abbr', 'Link', 'Sale Price', 'ROI',
       'Total Raise', 'Launchpad', 'Ended', 'When', 'Funds', 'Type', 'Raise',
       'Initial Cap', 'Market Cap', 'Blockchain', 'Category', 'Listing Date',
       'ATH ROI', 'ICO Days', 'Rounds', 'ICO Order', 'Social',
       'Funds and Investors', 'Investor Count', 'ICO Start', 'ICO End',
       'Soft Cap', 'Hard Cap', 'ICO Raised', 'Other Link', 'Success'],
      dtype='object')